In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from sklearn.linear_model import LinearRegression
from nba_api.stats.static import players
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt
import datetime as dt
import re

today = dt.date.today()
today_month = today.month
if (today_month > 7):
    current_year = today.year + 1
else:
    current_year = today.year

In [16]:
# Retreiving BOY projected VORPS
boy_vorps = pd.read_csv('In_Season/Data/opening_day_vorps_player.csv', index_col = 0)
# Changing Vorp_Projection column to be a float
boy_vorps['VORP_projection'] = boy_vorps.VORP_projection.apply(lambda x: float(x.strip('[]')) if type(x)==str else x)

# Retrieving current year VORPs
tables = pd.read_html(f'https://www.basketball-reference.com/leagues/NBA_{str(current_year)}_advanced.html')
table = tables[0]
table = table[['Player', 'Tm', 'G', 'VORP']]
table.columns = ['Player', 'Team', 'Games', 'VORP']
table = table[table.Team != 'Tm']
table = table[table.Team != 'TOT']
table['VORP'] = table.VORP.apply(pd.to_numeric)
table = table.groupby(['Player']).agg({'VORP' : 'sum', 'Games' : 'sum'})
table = pd.DataFrame(table)
table.reset_index(drop = False, inplace = True)
table.columns = ['Player', 'VORP', 'Games']

In [17]:
table

,Player,VORP,Games
0,Aaron Gordon,0.7,48
1,Aaron Henry,-0.1,6
2,Aaron Holiday,-0.1,40
3,Aaron Nesmith,-0.4,38
4,Aaron Wiggins,0.0,31
...,...,...,...
586,Zach Collins,0.0,1
587,Zach LaVine,1.8,44
588,Zeke Nnaji,0.1,37
589,Ziaire Williams,-0.4,35
